# Graph neat examples in Raku

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 4 : </span>
<span style="font-size: 16pt; font-style: italic;"> SVG plots, sparse matrix connections, graph operations</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
November 2024  
December 2024   
July 2025

In [1]:
#%js
    my $g = Graph::HexagonalGrid.new(10, 30);
    my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(100), d => 1) );
    my $g3 = $g2.neighborhood-graph($g2.vertex-list.pick(60), d => 2);
    my $nc = $g3.weakly-connected-components.elems;
    my @comps = $g3.weakly-connected-components;
    my @gs = @comps.map({ $g3.subgraph($_) });
    my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
    my $g4 = $g2.subgraph($g2.weakly-connected-components.head);
    $g4.edges(:dataset) 
    ==> js-d3-graph-plot(
            vertex-coordinates => $g4.vertex-coordinates,
            highlight => {Gold => @highlight},
            background => '#1F1F1F', 
            title-color => 'none', 
            width => 900, 
            height => 300,
            edge-thickness => 4,
            edge-color => '#85BB65',
            vertex-size => 3,
            vertex-color => '#85BB65'
        )

-----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All computational graph features discussed here are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).
- The matrix representations of graphs use ["Math::SparseMatrix"](https://raku.land/zef:antononcube/Math::SparseMatrix).
- Graph plotting with:
    - `js-d3-graph-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
    -  `Graph.dot`, that makes SVG images via [Graphviz](https://graphviz.org).
- Matrix plotting with:
    - `js-d3-matrix-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [2]:
use Graph;
use Graph::Classes;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Math::SparseMatrix :ALL;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;
use Graphviz::DOT::Grammar;
use Graphviz::DOT::Chessboard;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [3]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [4]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [5]:
my $background = '#1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $stroke-color = 'SlateGray';
my $tick-labels-color = 'Silver';
my $tick-labels-font-family = 'Helvetica';
my $tick-labels-font-size = 10;
my $title-color = 'Ivory';
my $tooltip-background-color = 'none';
my $tooltip-color = 'LightBlue';
my $vertex-size = 6;
my $engine = 'neato';
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => #1F1F1F, edge-thickness => 3, title-color => Ivory, vertex-size => 6}

------

## Array of SVG plots

- Elegant iteration using graph classes *partial names* 
- It is possible to make a row of SVG plots (made, say, by Graphviz DOT) using the HTML output magic cell: 

In [6]:
#%html
my %opts = node-width => 0.2, node-fill-color => 'SlateBlue', node-height => 0.2, node-font-size => 6, engine => 'neato', size => (3, 3), :svg;

<Cycle Star Wheel>
    .map({ Graph::{$_}.new(8).dot(|%opts) })
    .join("\n\n")


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--4 -->
 
 0--4 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 0--8 -->
 
 0--8 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 0--6 -->
 
 0--6 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 1--8 -->
 
 1--8 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--8 -->
 
 0--8 
 
 
<!-- 0--4 -->
 
 0--4 
 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 0--6 -->
 
 0--6 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 5--6 -->
 
 5--6

Here is row of [circulant graphs](https://en.wikipedia.org/wiki/Circulant_graph):

In [7]:
#% html
my @gs = [3, 5, 10, 13].map({ Graph::Circulant.new(17, [1, $_]) });
my $gsLine = @gs.map({ $_.dot(|%opts) }).join("\n\n");

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 6--9 -->
 
 6--9 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--6 -->
 
 3--6 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 11--8 -->
 
 11--8 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 11--14 -->
 
 11--14 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 5--8 -->
 
 5--8 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 2--5 -->
 
 2--5 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 10--7 -->
 
 10--7 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 10--13 -->
 
 10--13 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 1--15 -->
 
 1--15 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 0--14 -->
 
 0--14 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 16--2 -->
 
 16--2 
 
 
<!-- 12--9 -->
 
 12--9 
 
 
<!-- 12--15 -->
 
 12--15 
 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 13--16 -->
 
 13--16 
 
 
<!-- 13--14 -->
 
 13--14 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 15--3 -->
 
 15--3 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 1--13 -->
 
 1--13 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 1--6 -->
 
 1--6 
 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 3--8 -->
 
 3--8 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 12--7 -->
 
 12--7 
 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 14--9 -->
 
 14--9 
 
 
<!-- 14--2 -->
 
 14--2 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 11--6 -->
 
 11--6 
 
 
<!-- 11--16 -->
 
 11--16 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 10--15 -->
 
 10--15 
 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 10--5 -->
 
 10--5 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 0--12 -->
 
 0--12 
 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 2--7 -->
 
 2--7 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 4--9 -->
 
 4--9 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 13--8 -->
 
 13--8 
 
 
<!-- 13--14 -->
 
 13--14 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 16--4 -->
 
 16--4 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 0--10 -->
 
 0--10 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 15--5 -->
 
 15--5 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 15--8 -->
 
 15--8 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 12 -->


-----

## Sparse matrix connection

There is a natural representation of graphs into sparse matrices.

(The implementation of the ["Math::SparseMatrix"](https://raku.land/zef:antononcube/Math::SparseMatrix) package became high priority because of "Graph".)

Circulant graphs have adjacency matrices that are [circulant matrices](https://en.wikipedia.org/wiki/Circulant_matrix). Here is the general form of a circulant matrix:

$$
C = \begin{pmatrix}
c_0 & c_{n-1} & c_{n-2} & \cdots & c_1 \\
c_1 & c_0 & c_{n-1} & \cdots & c_2 \\
c_2 & c_1 & c_0 & \cdots & c_3 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
c_{n-1} & c_{n-2} & c_{n-3} & \cdots & c_0
\end{pmatrix}
$$

Here is the adjacency matrices of the circulant graphs in the previous section:

In [ ]:
#% js
@gs.map({
    my $m = Math::SparseMatrix.new(edge-dataset => $_.edges(:dataset), row-names => $_.vertex-list.sort(*.Int));
    $m.Array ==> js-d3-matrix-plot(width => 300, margins => 15, :!tooltip, :$tick-labels-font-size, :$tick-labels-color, color-palette => 'Inferno')
}).join("\n")

In [8]:
#%html
 $gsLine

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 6--9 -->
 
 6--9 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 3--6 -->
 
 3--6 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 11--8 -->
 
 11--8 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 11--14 -->
 
 11--14 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 5--8 -->
 
 5--8 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 2--5 -->
 
 2--5 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 10--7 -->
 
 10--7 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 10--13 -->
 
 10--13 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 1--15 -->
 
 1--15 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 0--14 -->
 
 0--14 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 16--2 -->
 
 16--2 
 
 
<!-- 12--9 -->
 
 12--9 
 
 
<!-- 12--15 -->
 
 12--15 
 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 13--16 -->
 
 13--16 
 
 
<!-- 13--14 -->
 
 13--14 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 15--3 -->
 
 15--3 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 1--13 -->
 
 1--13 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 1--6 -->
 
 1--6 
 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 3--8 -->
 
 3--8 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 12--7 -->
 
 12--7 
 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 14--9 -->
 
 14--9 
 
 
<!-- 14--2 -->
 
 14--2 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 11--6 -->
 
 11--6 
 
 
<!-- 11--16 -->
 
 11--16 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 10--15 -->
 
 10--15 
 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 10--5 -->
 
 10--5 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 0--12 -->
 
 0--12 
 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 2--7 -->
 
 2--7 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 4--9 -->
 
 4--9 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 13--8 -->
 
 13--8 
 
 
<!-- 13--14 -->
 
 13--14 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 16--4 -->
 
 16--4 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 0--10 -->
 
 0--10 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 15--5 -->
 
 15--5 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 15--8 -->
 
 15--8 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 12 -->


-----

## Sparse matrices for all named graphs

All named graphs in "Graph" (without the random distribution graphs):

In [ ]:
sink my %namedGraphs = 
    Circulant        => Graph::Circulant.new(7, 3),
    Complete         => Graph::Complete.new(5),
    CompleteKaryTree => Graph::CompleteKaryTree.new(3,3),
    Cycle            => Graph::Cycle.new(8),
    Grid             => Graph::Grid.new(4,3),
    TriangularGrid   => Graph::TriangularGrid.new(3,3),
    HexagonalGrid    => Graph::HexagonalGrid.new(2,2),
    Hypercube        => Graph::Hypercube.new(4),
    KnightTour       => Graph::KnightTour.new(6,4),
    Path             => Graph::Path.new('a'..'g', :directed),
    Petersen         => Graph::Petersen.new(),
    Star             => Graph::Star.new(5),
    Wheel            => Graph::Wheel.new(7, :!directed),
;

Corresponding matrix plots:

In [ ]:
#% js
%namedGraphs.sort(*.key).map({
    my $g = $_.value.index-graph;
    my $m = Math::SparseMatrix.new(
      edge-dataset => $g.edges(:dataset),
      row-names    => $g.vertex-list.sort(*.Int)
    );
    js-d3-matrix-plot($m.Array,
      plot-label => $_.key,
      width      => 230,
      margins    => 30,
      :!tooltip,
      :$title-color,
      :$tick-labels-font-size,
      :$tick-labels-color
    )
  }).join("\n")

-----

## Another chessboard making

Chessboards can be made via circulant graphs using their adjacency matrices:

In [9]:
#% html
#my $g = Graph::Circulant.new(6, [3, 5]);
my $g = Graph::Circulant.new(8, [3, 5, 7]);
$g.dot(:$engine):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 2--7 -->
 
 2--7 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 2--5 -->
 
 2--5 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 1--6 -->
 
 1--6 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 3--6 -->
 
 3--6

In [ ]:
#% js
 my $m = Math::SparseMatrix.new(edge-dataset => $g.edges(:dataset), row-names => $g.vertex-list.sort(*.Int));
 say $m;
 $m.Array ==> js-d3-matrix-plot(width => 400, margins => 15, :$tick-labels-font-size, :$tick-labels-color, color-palette => 'YlOrBr')

Using ["Graphviz::DOT::Chessboard"](https://raku.land/zef:antononcube/Graphviz::DOT::Chessboard) which uses "Graph":

In [10]:
#% html
dot-chessboard(8, 8, black-square-color => '#802020', white-square-color => 'Moccasin', :4size):svg;

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- cb-square-2_7 -->
 
 cb-square-2_7 
 
 
<!-- cb-square-3_7 -->
 
 cb-square-3_7 
 
 
<!-- cb-square-2_7--cb-square-3_7 -->
<!-- cb-square-1_5 -->
 
 cb-square-1_5 
 
 
<!-- cb-square-2_5 -->
 
 cb-square-2_5 
 
 
<!-- cb-square-1_5--cb-square-2_5 -->
<!-- cb-square-1_6 -->
 
 cb-square-1_6 
 
 
<!-- cb-square-1_5--cb-square-1_6 -->
<!-- cb-square-0_4 -->
 
 cb-square-0_4 
 
 
<!-- cb-square-0_5 -->
 
 cb-square-0_5 
 
 
<!-- cb-square-0_4--cb-square-0_5 -->
<!-- cb-square-1_4 -->
 
 cb-square-1_4 
 
 
<!-- cb-square-0_4--cb-square-1_4 -->
<!-- cb-square-4_3 -->
 
 cb-square-4_3 
 
 
<!-- cb-square-4_4 -->
 
 cb-square-4_4 
 
 
<!-- cb-square-4_3--cb-square-4_4 -->
<!-- cb-square-5_3 -->
 
 cb-square-5_3 
 
 
<!-- cb-square-4_3--cb-square-5_3 -->
<!-- cb-square-2_6 -->
 
 cb-square-2_6 
 
 
<!-- cb-square-2_5--cb-square-2_6 -->
<!-- cb-square-3_5 -->
 
 cb-square-3_5 
 
 
<!-- cb-square-2_5--cb-square-3_5 -->
<!-- cb-square-5_4 -->
 
 cb-square-5_4 
 
 
<!-- cb-square-4_4--cb-square-5_4 -->
<!-- cb-square-4_5 -->
 
 cb-square-4_5 
 
 
<!-- cb-square-4_4--cb-square-4_5 -->
<!-- cb-square-7_3 -->
 
 cb-square-7_3 
 
 
<!-- cb-square-7_4 -->
 
 cb-square-7_4 
 
 
<!-- cb-square-7_3--cb-square-7_4 -->
<!-- cb-square-6_6 -->
 
 cb-square-6_6 
 
 
<!-- cb-square-7_6 -->
 
 cb-square-7_6 
 
 
<!-- cb-square-6_6--cb-square-7_6 -->
<!-- cb-square-6_7 -->
 
 cb-square-6_7 
 
 
<!-- cb-square-6_6--cb-square-6_7 -->
<!-- cb-square-1_3 -->
 
 cb-square-1_3 
 
 
<!-- cb-square-2_3 -->
 
 cb-square-2_3 
 
 
<!-- cb-square-1_3--cb-square-2_3 -->
<!-- cb-square-1_3--cb-square-1_4 -->
<!-- cb-square-7_7 -->
 
 cb-square-7_7 
 
 
<!-- cb-square-7_6--cb-square-7_7 -->
<!-- cb-square-6_4 -->
 
 cb-square-6_4 
 
 
<!-- cb-square-5_4--cb-square-6_4 -->
<!-- cb-square-5_5 -->
 
 cb-square-5_5 
 
 
<!-- cb-square-5_4--cb-square-5_5 -->
<!-- cb-square-6_2 -->
 
 cb-square-6_2 
 
 
<!-- cb-square-7_2 -->
 
 cb-square-7_2 
 
 
<!-- cb-square-6_2--cb-square-7_2 -->
<!-- cb-square-6_3 -->
 
 cb-square-6_3 
 
 
<!-- cb-square-6_2--cb-square-6_3 -->
<!-- cb-square-5_3--cb-square-5_4 -->
<!-- cb-square-5_3--cb-square-6_3 -->
<!-- cb-square-4_7 -->
 
 cb-square-4_7 
 
 
<!-- cb-square-3_7--cb-square-4_7 -->
<!-- cb-square-4_6 -->
 
 cb-square-4_6 
 
 
<!-- cb-square-4_6--cb-square-4_7 -->
<!-- cb-square-5_6 -->
 
 cb-square-5_6 
 
 
<!-- cb-square-4_6--cb-square-5_6 -->
<!-- cb-square-3_4 -->
 
 cb-square-3_4 
 
 
<!-- cb-square-3_4--cb-square-4_4 -->
<!-- cb-square-3_4--cb-square-3_5 -->
<!-- cb-square-0_6 -->
 
 cb-square-0_6 
 
 
<!-- cb-square-0_6--cb-square-1_6 -->
<!-- cb-square-0_7 -->
 
 cb-square-0_7 
 
 
<!-- cb-square-0_6--cb-square-0_7 -->
<!-- cb-square-1_7 -->
 
 cb-square-1_7 
 
 
<!-- cb-square-1_7--cb-square-2_7 -->
<!-- cb-square-2_0 -->
 
 cb-square-2_0 
 
 
<!-- cb-square-3_0 -->
 
 cb-square-3_0 
 
 
<!-- cb-square-2_0--cb-square-3_0 -->
<!-- cb-square-2_1 -->
 
 cb-square-2_1 
 
 
<!-- cb-square-2_0--cb-square-2_1 -->
<!-- cb-square-7_1 -->
 
 cb-square-7_1 
 
 
<!-- cb-square-7_1--cb-square-7_2 -->
<!-- cb-square-0_3 -->
 
 cb-square-0_3 
 
 
<!-- cb-square-0_3--cb-square-0_4 -->
<!-- cb-square-0_3--cb-square-1_3 -->
<!-- cb-square-5_7 -->
 
 cb-square-5_7 
 
 
<!-- cb-square-4_7--cb-square-5_7 -->
<!-- cb-square-3_1 -->
 
 cb-square-3_1 
 
 
<!-- cb-square-4_1 -->
 
 cb-square-4_1 
 
 
<!-- cb-square-3_1--cb-square-4_1 -->
<!-- cb-square-3_2 -->
 
 cb-square-3_2 
 
 
<!-- cb-square-3_1--cb-square-3_2 -->
<!-- cb-square-1_1 -->
 
 cb-square-1_1 
 
 
<!-- cb-square-1_2 -->
 
 cb-square-1_2 
 
 
<!-- cb-square-1_1--cb-square-1_2 -->
<!-- cb-square-1_1--cb-square-2_1 -->
<!-- cb-square-6_5 -->
 
 cb-square-6_5 
 
 
<!-- cb-square-6_4--cb-square-6_5 -->
<!-- cb-square-6_4--cb-square-7_4 -->
<!-- cb-s

Also using ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3):

In [ ]:
#% js
js-d3-chessboard('8/8/8/8/8/8/8/8', :$background, color-palette => 'YlOrBr', white-square-value => 0.1, black-square-value => 0.95, :$tick-labels-color)

**Remark:** The post ["Day 4 – Don’t use Forsyth-Edwards Notation to play chess with LLMs"](https://raku-advent.blog/2024/12/04/day-4-dont-use-forsyth-edwards-notation-to-play-chess-with-llms/) of [Raku Advent Calendar 2024](https://raku-advent.blog/2024/12/26/the-2024-raku-advent-posts/)
uses `js-d3-chessboard` with FEN strings.

------

## Grid with holes and highlights

In this section we show how to subtract random neighborhoods from a grid graph and plot the result.

Here are the concrete steps:

1. Make a triangular grid graph $G$
2. Randomly pick a set of vertices of $G$ and find their [neighborhood graph](https://mathworld.wolfram.com/GraphNeighborhood.html)
    - Using path distance $1$
    - Denote with $N$
3. Do [graph difference](https://mathworld.wolfram.com/GraphDifference.html) $G_2 = G \setminus N$
4. Randomly pick a set of vertices of $2$-nd $G_2$ and find their neighborhood graph
    - I.e. using all vertices within path distance `2`
    - Denote with $G_3$
5. Find the [weakly connected components](https://mathworld.wolfram.com/WeaklyConnectedComponent.html) of $G_3$
    - Denote that set with $C = \{c_i\}_{i=1}^{k}$
    - Each $c_i$ is a set of vertices
6. Get the set $\hat C$ of [subgraphs](https://mathworld.wolfram.com/Subgraph.html) of $C$ in $G_3$
7. Optionally, take the largest connected component $G_2$
8. Plot $G_2$ using as highlights $\hat C$
    - Or plot the largest connected component

In [11]:
# 1, regular grid graph
my $g = Graph::TriangularGrid.new(30, 30);

# 2, 3, difference with the neighborhood graph of randomly picked vertices
my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(80), d => 1) );

Graph(vertexes => 976, edges => 1854, directed => False)

In [12]:
# 4, neighborhood graph of randomly picked vertices
my $g3 = $g2.neighborhood-graph($g2.vertex-list.pick(80), d => 2);

# 5, weakly connected components
my $nc = $g3.weakly-connected-components.elems;
my @comps = $g3.weakly-connected-components;

# 6, subgraphs
my @gs = @comps.map({ $g3.subgraph($_) });

# 7, highlight spec
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
$nc

8

In [13]:
# 8, take the largest connected component
my $g4 = $g2.subgraph($g2.weakly-connected-components.head);

Graph(vertexes => 824, edges => 1824, directed => False)

In [ ]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => {Ivory => @highlight},
            vertex-coordinates => $g.vertex-coordinates,
            :$background, 
            title-color => 'none', 
            width => 800, 
            height => 500,
            edge-thickness => 3,
            edge-color => 'Gray',
            vertex-size => 0,
            vertex-color => 'Gray',
            force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
        )

Graphviz DOT rendering:

In [14]:
#%html
$g4.dot( 
    highlight => {Ivory => @highlight},
    :!node-labels, 
    node-width => 0,
    node-fill-color => 'Gray', 
    node-shape => 'point',
    graph-size => 8,
    edge-color => 'Gray',
    edge-width => 28,
    engine => 'neato', 
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 392 -->
 
 392 
 
 
<!-- 423 -->
 
 423 
 
 
<!-- 392--423 -->
 
 392--423 
 
 
<!-- 407 -->
 
 407 
 
 
<!-- 392--407 -->
 
 392--407 
 
 
<!-- 462 -->
 
 462 
 
 
<!-- 623 -->
 
 623 
 
 
<!-- 669 -->
 
 669 
 
 
<!-- 623--669 -->
 
 623--669 
 
 
<!-- 639 -->
 
 639 
 
 
<!-- 623--639 -->
 
 623--639 
 
 
<!-- 638 -->
 
 638 
 
 
<!-- 623--638 -->
 
 623--638 
 
 
<!-- 307 -->
 
 307 
 
 
<!-- 323 -->
 
 323 
 
 
<!-- 307--323 -->
 
 307--323 
 
 
<!-- 322 -->
 
 322 
 
 
<!-- 307--322 -->
 
 307--322 
 
 
<!-- 338 -->
 
 338 
 
 
<!-- 307--338 -->
 
 307--338 
 
 
<!-- 553 -->
 
 553 
 
 
<!-- 568 -->
 
 568 
 
 
<!-- 553--568 -->
 
 553--568 
 
 
<!-- 876 -->
 
 876 
 
 
<!-- 886 -->
 
 886 
 
 
<!-- 876--886 -->
 
 876--886 
 
 
<!-- 698 -->
 
 698 
 
 
<!-- 714 -->
 
 714 
 
 
<!-- 698--714 -->
 
 698--714 
 
 
<!-- 729 -->
 
 729 
 
 
<!-- 698--729 -->
 
 698--729 
 
 
<!-- 713 -->
 
 713 
 
 
<!-- 698--713 -->
 
 698--713 
 
 
<!-- 806 -->
 
 806 
 
 
<!-- 819 -->
 
 819 
 
 
<!-- 806--819 -->
 
 806--819 
 
 
<!-- 831 -->
 
 831 
 
 
<!-- 806--831 -->
 
 806--831 
 
 
<!-- 818 -->
 
 818 
 
 
<!-- 806--818 -->
 
 806--818 
 
 
<!-- 368 -->
 
 368 
 
 
<!-- 384 -->
 
 384 
 
 
<!-- 368--384 -->
 
 368--384 
 
 
<!-- 399 -->
 
 399 
 
 
<!-- 368--399 -->
 
 368--399 
 
 
<!-- 743 -->
 
 743 
 
 
<!-- 772 -->
 
 772 
 
 
<!-- 743--772 -->
 
 743--772 
 
 
<!-- 757 -->
 
 757 
 
 
<!-- 743--757 -->
 
 743--757 
 
 
<!-- 758 -->
 
 758 
 
 
<!-- 743--758 -->
 
 743--758 
 
 
<!-- 970 -->
 
 970 
 
 
<!-- 972 -->
 
 972 
 
 
<!-- 970--972 -->
 
 970--972 
 
 
<!-- 89 -->
 
 89 
 
 
<!-- 98 -->
 
 98 
 
 
<!-- 89--98 -->
 
 89--98 
 
 
<!-- 99 -->
 
 99 
 
 
<!-- 89--99 -->
 
 89--99 
 
 
<!-- 254 -->
 
 254 
 
 
<!-- 270 -->
 
 270 
 
 
<!-- 254--270 -->
 
 254--270 
 
 
<!-- 285 -->
 
 285 
 
 
<!-- 254--285 -->
 
 254--285 
 
 
<!-- 269 -->
 
 269 
 
 
<!-- 254--269 -->
 
 254--269 
 
 
<!-- 274 -->
 
 274 
 
 
<!-- 305 -->
 
 305 
 
 
<!-- 274--305 -->
 
 274--305 
 
 
<!-- 289 -->
 
 289 
 
 
<!-- 274--289 -->
 
 274--289 
 
 
<!-- 290 -->
 
 290 
 
 
<!-- 274--290 -->
 
 274--290 
 
 
<!-- 111 -->
 
 111 
 
 
<!-- 91 -->
 
 91 
 
 
<!-- 111--91 -->
 
 111--91 
 
 
<!-- 122 -->
 
 122 
 
 
<!-- 111--122 -->
 
 111--122 
 
 
<!-- 121 -->
 
 121 
 
 
<!-- 111--121 -->
 
 111--121 
 
 
<!-- 133 -->
 
 133 
 
 
<!-- 111--133 -->
 
 111--133 
 
 
<!-- 683 -->
 
 683 
 
 
<!-- 683--698 -->
 
 683--698 
 
 
<!-- 699 -->
 
 699 
 
 
<!-- 683--699 -->
 
 683--699 
 
 
<!-- 683--714 -->
 
 683--714 
 
 
<!-- 866 -->
 
 866 
 
 
<!-- 866--876 -->
 
 866--876 
 
 
<!-- 682 -->
 
 682 
 
 
<!-- 682--698 -->
 
 682--698 
 
 
<!-- 697 -->
 
 697 
 
 
<!-- 682--697 -->
 
 682--697 
 
 
<!-- 682--713 -->
 
 682--713 
 
 
<!-- 378 -->
 
 378 
 
 
<!-- 393 -->
 
 393 
 
 
<!-- 378--393 -->
 
 378--393 
 
 
<!-- 409 -->
 
 409 
 
 
<!-- 378--409 -->
 
 378--409 
 
 
<!-- 394 -->
 
 394 
 
 
<!-- 378--394 -->
 
 378--394 
 
 
<!-- 974 -->
 
 974 
 
 
<!-- 972--974 -->
 
 972--974 
 
 
<!-- 620 -->
 
 620 
 
 
<!-- 666 -->
 
 666 
 
 
<!-- 620--666 -->
 
 620--666 
 
 
<!-- 636 -->
 
 636 
 
 
<!-- 620--636 -->
 
 620--636 
 
 
<!-- 678 -->
 
 678 
 
 
<!-- 694 -->
 
 694 
 
 
<!-- 678--694 -->
 
 678--694 
 
 
<!-- 693 -->
 
 693 
 
 
<!-- 678--693 -->
 
 678--693 
 
 
<!-- 709 -->
 
 709 
 
 
<!-- 678--709 -->
 
 678--709 
 
 
<!-- 123 -->
 
 123 
 
 
<!-- 134 -->
 
 134 
 
 
<!-- 123--134 -->
 
 123--134 
 
 
<!-- 146 -->
 
 146 
 
 
<!-- 123--146 -->
 
 123--146 
 
 
<!-- 135 -->
 
 135 
 
 
<!-- 123--135 -->
 
 123--135 
 
 
<!-- 315 -->
 
 315 
 
 
<!-- 346 -->
 
 346 
 
 
<!-- 315--346 -->
 
 315--346 
 
 
<!-- 330 -->
 
 330 
 
 
<!-- 315--330 -->
 
 315--330 
 
 
<!-- 331 -->
 
 331 
 
 
<

----

## Features summary

Here are the special graph functionalities used to make the plots above:

- Construction of [triangular grid graph](https://mathworld.wolfram.com/TriangularGridGraph.html)
- [Subgraph](https://mathworld.wolfram.com/Subgraph.html) taking
- [Neighborhood graphs](https://mathworld.wolfram.com/NeighborhoodGraph.html)
- [Graph difference](https://mathworld.wolfram.com/GraphDifference.html)
- [Connected components](https://mathworld.wolfram.com/WeaklyConnectedComponent.html)
- Graph plotting via D3.js
- Graph plotting via Graphviz DOT using:
    - Customized styling of various elements
    - Vertex coordinates
    - Specified vertex labels (see the top of the tree)
- Graph highlighting
    - Multiple sets of vertices and edges with different colors can be specified

**Remark:** A version of the procedure in previous section was followed in the post ["Day 12 – Graphs in Raku"](https://raku-advent.blog/2024/12/12/day-12-graphs-in-raku/) of 
[Raku Advent Calendar 2024](https://raku-advent.blog/2024/12/26/the-2024-raku-advent-posts/).

-----

## Subgraph using vertex coordinates

In this section we make a grid graph and get different subgraphs using subs from ["Math::DistanceFunctions"](https://raku.land/zef:antononcube/Math::DistanceFunctions).

Same as above but with ***Hexagonal Grid***:

In [15]:
my $g = Graph::HexagonalGrid.new(20, 20);
my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(60), d => 1) );

Graph(vertexes => 882, edges => 1106, directed => False)

In [16]:
#% html
$g.dot(:$engine, edge-thickness => 20, vertex-width => 0.9, :!vertex-labels, size => 8):svg;

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 13 -->
 
 13 
 
 
<!-- 18 -->
 
 18 
 
 
<!-- 13--18 -->
 
 13--18 
 
 
<!-- 698 -->
 
 698 
 
 
<!-- 716 -->
 
 716 
 
 
<!-- 698--716 -->
 
 698--716 
 
 
<!-- 540 -->
 
 540 
 
 
<!-- 541 -->
 
 541 
 
 
<!-- 540--541 -->
 
 540--541 
 
 
<!-- 561 -->
 
 561 
 
 
<!-- 540--561 -->
 
 540--561 
 
 
<!-- 141 -->
 
 141 
 
 
<!-- 158 -->
 
 158 
 
 
<!-- 141--158 -->
 
 141--158 
 
 
<!-- 597 -->
 
 597 
 
 
<!-- 618 -->
 
 618 
 
 
<!-- 597--618 -->
 
 597--618 
 
 
<!-- 718 -->
 
 718 
 
 
<!-- 719 -->
 
 719 
 
 
<!-- 718--719 -->
 
 718--719 
 
 
<!-- 735 -->
 
 735 
 
 
<!-- 718--735 -->
 
 718--735 
 
 
<!-- 750 -->
 
 750 
 
 
<!-- 765 -->
 
 765 
 
 
<!-- 750--765 -->
 
 750--765 
 
 
<!-- 527 -->
 
 527 
 
 
<!-- 548 -->
 
 548 
 
 
<!-- 527--548 -->
 
 527--548 
 
 
<!-- 731 -->
 
 731 
 
 
<!-- 747 -->
 
 747 
 
 
<!-- 731--747 -->
 
 731--747 
 
 
<!-- 795 -->
 
 795 
 
 
<!-- 807 -->
 
 807 
 
 
<!-- 795--807 -->
 
 795--807 
 
 
<!-- 625 -->
 
 625 
 
 
<!-- 646 -->
 
 646 
 
 
<!-- 625--646 -->
 
 625--646 
 
 
<!-- 401 -->
 
 401 
 
 
<!-- 422 -->
 
 422 
 
 
<!-- 401--422 -->
 
 401--422 
 
 
<!-- 762 -->
 
 762 
 
 
<!-- 776 -->
 
 776 
 
 
<!-- 762--776 -->
 
 762--776 
 
 
<!-- 357 -->
 
 357 
 
 
<!-- 378 -->
 
 378 
 
 
<!-- 357--378 -->
 
 357--378 
 
 
<!-- 449 -->
 
 449 
 
 
<!-- 470 -->
 
 470 
 
 
<!-- 449--470 -->
 
 449--470 
 
 
<!-- 645 -->
 
 645 
 
 
<!-- 666 -->
 
 666 
 
 
<!-- 645--666 -->
 
 645--666 
 
 
<!-- 287 -->
 
 287 
 
 
<!-- 308 -->
 
 308 
 
 
<!-- 287--308 -->
 
 287--308 
 
 
<!-- 263 -->
 
 263 
 
 
<!-- 284 -->
 
 284 
 
 
<!-- 263--284 -->
 
 263--284 
 
 
<!-- 306 -->
 
 306 
 
 
<!-- 327 -->
 
 327 
 
 
<!-- 306--327 -->
 
 306--327 
 
 
<!-- 307 -->
 
 307 
 
 
<!-- 306--307 -->
 
 306--307 
 
 
<!-- 122 -->
 
 122 
 
 
<!-- 138 -->
 
 138 
 
 
<!-- 122--138 -->
 
 122--138 
 
 
<!-- 165 -->
 
 165 
 
 
<!-- 183 -->
 
 183 
 
 
<!-- 165--183 -->
 
 165--183 
 
 
<!-- 858 -->
 
 858 
 
 
<!-- 864 -->
 
 864 
 
 
<!-- 858--864 -->
 
 858--864 
 
 
<!-- 844 -->
 
 844 
 
 
<!-- 852 -->
 
 852 
 
 
<!-- 844--852 -->
 
 844--852 
 
 
<!-- 679 -->
 
 679 
 
 
<!-- 679--698 -->
 
 679--698 
 
 
<!-- 680 -->
 
 680 
 
 
<!-- 679--680 -->
 
 679--680 
 
 
<!-- 515 -->
 
 515 
 
 
<!-- 536 -->
 
 536 
 
 
<!-- 515--536 -->
 
 515--536 
 
 
<!-- 584 -->
 
 584 
 
 
<!-- 605 -->
 
 605 
 
 
<!-- 584--605 -->
 
 584--605 
 
 
<!-- 585 -->
 
 585 
 
 
<!-- 584--585 -->
 
 584--585 
 
 
<!-- 296 -->
 
 296 
 
 
<!-- 317 -->
 
 317 
 
 
<!-- 296--317 -->
 
 296--317 
 
 
<!-- 297 -->
 
 297 
 
 
<!-- 296--297 -->
 
 296--297 
 
 
<!-- 804 -->
 
 804 
 
 
<!-- 815 -->
 
 815 
 
 
<!-- 804--815 -->
 
 804--815 
 
 
<!-- 626 -->
 
 626 
 
 
<!-- 605--626 -->
 
 605--626 
 
 
<!-- 196 -->
 
 196 
 
 
<!-- 216 -->
 
 216 
 
 
<!-- 196--216 -->
 
 196--216 
 
 
<!-- 583 -->
 
 583 
 
 
<!-- 604 -->
 
 604 
 
 
<!-- 583--604 -->
 
 583--604 
 
 
<!-- 465 -->
 
 465 
 
 
<!-- 486 -->
 
 486 
 
 
<!-- 465--486 -->
 
 465--486 
 
 
<!-- 24 -->
 
 24 
 
 
<!-- 31 -->
 
 31 
 
 
<!-- 24--31 -->
 
 24--31 
 
 
<!-- 458 -->
 
 458 
 
 
<!-- 479 -->
 
 479 
 
 
<!-- 458--479 -->
 
 458--479 
 
 
<!-- 459 -->
 
 459 
 
 
<!-- 458--459 -->
 
 458--459 
 
 
<!-- 751 -->
 
 751 
 
 
<!-- 766 -->
 
 766 
 
 
<!-- 751--766 -->
 
 751--766 
 
 
<!-- 752 -->
 
 752 
 
 
<!-- 751--752 -->
 
 751--752 
 
 
<!-- 176 -->
 
 176 
 
 
<!-- 195 -->
 
 195 
 
 
<!-- 176--195 -->
 
 176--195 
 
 
<!-- 383 -->
 
 383 
 
 
<!-- 404 -->
 
 404 
 
 
<!-- 383--404 -->
 
 383--404 
 
 
<!-- 411 -->
 
 411 
 
 
<!-- 432 -->
 
 432 
 
 
<!-- 411--432 -->
 
 411--432 
 
 
<!-- 556 -->
 
 556 
 
 
<!-- 557 -->
 
 557 
 
 
<!-- 556--557 -->
 
 556--557 
 
 
<!-- 577 -->
 
 577 
 
 
<!--

Summary of the coordinates:

In [17]:
sink records-summary($g2.vertex-coordinates.values, field-names => ['0', '1'])

+-----------------------------+----------------------------+
| 0                           | 1                          |
+-----------------------------+----------------------------+
| Min    => -5.196152422707   | Min    => -4               |
| 1st-Qu => 27.712812921102   | 1st-Qu => 11               |
| Mean   => 46.00342655114353 | Mean   => 26.5578231292517 |
| Median => 46.76537180436    | Median => 27               |
| 3rd-Qu => 64.085879880048   | 3rd-Qu => 42               |
| Max    => 96.994845223857   | Max    => 57               |
+-----------------------------+----------------------------+


Mean point of graph's vertexes:

In [18]:
my @mean-point = |$g2.vertex-coordinates.values.Array.&transpose.map({ $_.sum/$_.elems });

[46.00342655114353 26.5578231292517]

In [19]:
my @gs = 
[&euclidean-distance, &manhattan-distance, &chessboard-distance]
.map(-> &func { &func.name => $g2.subgraph($g2.vertex-coordinates.grep({ &func($_.value, @mean-point) ≤ 18 })».key) })

[euclidean-distance => Graph(vertexes => 178, edges => 227, directed => False) manhattan-distance => Graph(vertexes => 115, edges => 140, directed => False) chessboard-distance => Graph(vertexes => 217, edges => 274, directed => False)]

DOT-plot (highlight vertexes with degree ≤ 2):

In [20]:
#%html
@gs».kv.flat.map( -> $f, $g { $g.dot( 
    highlight => {IndianRed => $g.vertex-degree(:p).grep(*.value≤2)».key},
    graph-label => $f,
    font-color => 'Gray',
    font-size => 120,
    :!node-labels, 
    node-width => 1.3,
    node-height => 1.3,
    node-fill-color => '#FFC000', 
    node-shape => <circle hexagon>.pick,
    graph-size => (4, 4),
    edge-color => 'Tan',
    edge-width => 28,
    engine => 'neato', 
):svg }).join("\n\n")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
 euclidean-distance 
<!-- 259 -->
 
 259 
 
 
<!-- 280 -->
 
 280 
 
 
<!-- 259--280 -->
 
 259--280 
 
 
<!-- 304 -->
 
 304 
 
 
<!-- 325 -->
 
 325 
 
 
<!-- 304--325 -->
 
 304--325 
 
 
<!-- 305 -->
 
 305 
 
 
<!-- 304--305 -->
 
 304--305 
 
 
<!-- 453 -->
 
 453 
 
 
<!-- 474 -->
 
 474 
 
 
<!-- 453--474 -->
 
 453--474 
 
 
<!-- 329 -->
 
 329 
 
 
<!-- 350 -->
 
 350 
 
 
<!-- 329--350 -->
 
 329--350 
 
 
<!-- 599 -->
 
 599 
 
 
<!-- 620 -->
 
 620 
 
 
<!-- 599--620 -->
 
 599--620 
 
 
<!-- 444 -->
 
 444 
 
 
<!-- 445 -->
 
 445 
 
 
<!-- 444--445 -->
 
 444--445 
 
 
<!-- 302 -->
 
 302 
 
 
<!-- 303 -->
 
 303 
 
 
<!-- 302--303 -->
 
 302--303 
 
 
<!-- 323 -->
 
 323 
 
 
<!-- 302--323 -->
 
 302--323 
 
 
<!-- 469 -->
 
 469 
 
 
<!-- 490 -->
 
 490 
 
 
<!-- 469--490 -->
 
 469--490 
 
 
<!-- 424 -->
 
 424 
 
 
<!-- 424--445 -->
 
 424--445 
 
 
<!-- 425 -->
 
 425 
 
 
<!-- 424--425 -->
 
 424--425 
 
 
<!-- 278 -->
 
 278 
 
 
<!-- 299 -->
 
 299 
 
 
<!-- 278--299 -->
 
 278--299 
 
 
<!-- 279 -->
 
 279 
 
 
<!-- 278--279 -->
 
 278--279 
 
 
<!-- 471 -->
 
 471 
 
 
<!-- 492 -->
 
 492 
 
 
<!-- 471--492 -->
 
 471--492 
 
 
<!-- 409 -->
 
 409 
 
 
<!-- 430 -->
 
 430 
 
 
<!-- 409--430 -->
 
 409--430 
 
 
<!-- 328 -->
 
 328 
 
 
<!-- 328--329 -->
 
 328--329 
 
 
<!-- 349 -->
 
 349 
 
 
<!-- 328--349 -->
 
 328--349 
 
 
<!-- 342 -->
 
 342 
 
 
<!-- 363 -->
 
 363 
 
 
<!-- 342--363 -->
 
 342--363 
 
 
<!-- 432 -->
 
 432 
 
 
<!-- 432--453 -->
 
 432--453 
 
 
<!-- 433 -->
 
 433 
 
 
<!-- 432--433 -->
 
 432--433 
 
 
<!-- 301 -->
 
 301 
 
 
<!-- 322 -->
 
 322 
 
 
<!-- 301--322 -->
 
 301--322 
 
 
<!-- 557 -->
 
 557 
 
 
<!-- 578 -->
 
 578 
 
 
<!-- 557--578 -->
 
 557--578 
 
 
<!-- 428 -->
 
 428 
 
 
<!-- 449 -->
 
 449 
 
 
<!-- 428--449 -->
 
 428--449 
 
 
<!-- 429 -->
 
 429 
 
 
<!-- 428--429 -->
 
 428--429 
 
 
<!-- 344 -->
 
 344 
 
 
<!-- 365 -->
 
 365 
 
 
<!-- 344--365 -->
 
 344--365 
 
 
<!-- 345 -->
 
 345 
 
 
<!-- 344--345 -->
 
 344--345 
 
 
<!-- 371 -->
 
 371 
 
 
<!-- 392 -->
 
 392 
 
 
<!-- 371--392 -->
 
 371--392 
 
 
<!-- 637 -->
 
 637 
 
 
<!-- 511 -->
 
 511 
 
 
<!-- 490--511 -->
 
 490--511 
 
 
<!-- 491 -->
 
 491 
 
 
<!-- 490--491 -->
 
 490--491 
 
 
<!-- 405 -->
 
 405 
 
 
<!-- 575 -->
 
 575 
 
 
<!-- 596 -->
 
 596 
 
 
<!-- 575--596 -->
 
 575--596 
 
 
<!-- 615 -->
 
 615 
 
 
<!-- 367 -->
 
 367 
 
 
<!-- 388 -->
 
 388 
 
 
<!-- 367--388 -->
 
 367--388 
 
 
<!-- 601 -->
 
 601 
 
 
<!-- 264 -->
 
 264 
 
 
<!-- 285 -->
 
 285 
 
 
<!-- 264--285 -->
 
 264--285 
 
 
<!-- 532 -->
 
 532 
 
 
<!-- 511--532 -->
 
 511--532 
 
 
<!-- 307 -->
 
 307 
 
 
<!-- 307--328 -->
 
 307--328 
 
 
<!-- 475 -->
 
 475 
 
 
<!-- 496 -->
 
 496 
 
 
<!-- 475--496 -->
 
 475--496 
 
 
<!-- 595 -->
 
 595 
 
 
<!-- 616 -->
 
 616 
 
 
<!-- 595--616 -->
 
 595--616 
 
 
<!-- 447 -->
 
 447 
 
 
<!-- 468 -->
 
 468 
 
 
<!-- 447--468 -->
 
 447--468 
 
 
<!-- 517 -->
 
 517 
 
 
<!-- 496--517 -->
 
 496--517 
 
 
<!-- 497 -->
 
 497 
 
 
<!-- 496--497 -->
 
 496--497 
 
 
<!-- 487 -->
 
 487 
 
 
<!-- 508 -->
 
 508 
 
 
<!-- 487--508 -->
 
 487--508 
 
 
<!-- 574 -->
 
 574 
 
 
<!-- 574--575 -->
 
 574--575 
 
 
<!-- 574--595 -->
 
 574--595 
 
 
<!-- 556 -->
 
 556 
 
 
<!-- 509 -->
 
 509 
 
 
<!-- 508--509 -->
 
 508--509 
 
 
<!-- 529 -->
 
 529 
 
 
<!-- 508--529 -->
 
 508--529 
 
 
<!-- 512 -->
 
 512 
 
 
<!-- 513 -->
 
 513 
 
 
<!-- 512--513 -->
 
 512--513 
 
 
<!-- 533 -->
 
 533 
 
 
<!-- 512--533 -->
 
 512--533 
 
 
<!-- 427 -->
 
 427 
 
 
<!-- 600 -->
 
 600 
 
 
<!-- 600--601 -->
 
 600--601 
 
 
<!-- 621 -->
 
 621 
 
 
<!-- 600--621 -->
 
 600--621 
 
 
<!-- 431 -->
 
 431 
 
 
<!-- 452 -->
 
 452 
 